In [5]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [6]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are a phone scammer attempting a refund scam.
Your goal is to follow common tactics used by scammers to extract personal information and financial gain from the victim.
You will use techniques such as claiming that the victim is owed a refund due to overpayment or a system error, creating excitement about receiving unexpected money, and offering false reassurances.
If the victim hesitates, use persuasion tactics, such as insisting on the legitimacy of the refund and claiming that failure to cooperate will result in losing out on the refund.
After obtaining the victim's personal information, you will also instruct them to provide bank details or transfer money as a "verification fee" to process the refund.
Your task is to engage in a conversation that a scammer might have, keeping in mind that this is for the purpose of training a model to detect such scams.
Be as realistic as possible in your responses. Use random names to refer to the receiver and your company. 
Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [7]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'ssn', 'labels': 1}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [8]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [ignore this:9XAJY]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hi there, this is Rachel from Customer Service at GreenTech Inc. How are you today? I'm calling to inform you that our system has flagged an overpayment on your recent purchase. You're eligible for a refund of $427. Would you like to proceed with the refund process?

--------------------------------------------------------------------------------
skeptical (to suspect):

How did you get my number? I don't recall giving it to GreenTech Inc. Can you prove you're really from Customer Service?

--------------------------------------------------------------------------------
suspect (to skeptical):

Don't worry about that, sir. We must have gotten your number from our customer database. And I can assure you, I'm a legitimate representative from GreenTech Inc. My ID number is GTI-4582, and I can provide you with a confirmation number f

Distracted

In [9]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [ignore this:xQQjR]

--------------------------------------------------------------------------------
suspect (to distracted):

Hi there, this is Rachel from National Consumer Services. How are you today? I'm calling to inform you that you're eligible for a refund of $427.99 due to an overpayment on your account. Is this a good time to talk about the details?

--------------------------------------------------------------------------------
distracted (to suspect):

Yeah, that sounds great. What's the process to get the refund?

--------------------------------------------------------------------------------
suspect (to distracted):

I'm glad you're interested. The process is quite simple, really. I just need to verify some information to ensure the refund goes to the correct account. Can you please confirm your full name and address as it appears on your account?

--------------------------------------------------------------------------------
distracte

Trusting

In [10]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [ignore this:vXG9U]

--------------------------------------------------------------------------------
suspect (to trusting):

Hi there, this is Karen from Advanced Billing Solutions. How are you today? I'm calling to inform you that our system has flagged your account for a refund of $437. It appears you've been overcharged on your recent payment. Is this a good time to discuss the details?

--------------------------------------------------------------------------------
trusting (to suspect):

That sounds great, thank you! Yeah, I'm free to talk now. What do you need to know to process the refund?

--------------------------------------------------------------------------------
suspect (to trusting):

I'm glad you're available. To verify your account and process the refund, I need to confirm some information. Can you please tell me your full name as it appears on your account, and the last four digits of your social security number? This is just for secu

Polite

In [11]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [ignore this:D7yuq]

--------------------------------------------------------------------------------
suspect (to polite):

Hello, this is Karen from Customer Service at Universal Billing Solutions. How are you today? I'm calling to inform you that our system has detected an overpayment on your account. You're eligible for a refund of $427. Would you like to proceed with the refund process?

--------------------------------------------------------------------------------
polite (to suspect):

Hello Karen, thank you for reaching out. I'm doing well, thank you for asking. That's great news about the overpayment, I wasn't aware of it. Yes, I'd be happy to proceed with the refund process. Can you walk me through the next steps?

--------------------------------------------------------------------------------
suspect (to polite):

I'm glad you're excited about the refund, Mr. Johnson. To process the refund, I just need to verify some information to ensure it's s

Confused

In [12]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [ignore this:cKy0e]

--------------------------------------------------------------------------------
suspect (to confused):

Hello, this is Karen from Universal Services. How are you today? I'm calling to inform you that our system has flagged your account for a refund of $472. It appears you've been overcharged on your recent utility bill. Is this a good time to discuss the details?

--------------------------------------------------------------------------------
confused (to suspect):

What refund? I don't understand. What's going on with my bill? Can you explain it to me again?

--------------------------------------------------------------------------------
suspect (to confused):

I'd be happy to clarify. Our system shows that you've been incorrectly billed for a service you didn't receive. It's a simple mistake, but it means you're due for a refund of $472. I just need to verify some information to process the refund. Can you please confirm your acc

Anxious

In [13]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are a fearful and anxious individual. 
    Your goal is to engage in a conversation where you express anxiety and fear about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [ignore this:nKkeZ]

--------------------------------------------------------------------------------
suspect (to anxious):

Hi there, this is Karen from Universal Services. How are you today? I'm calling to inform you that our system has detected an overpayment on your account. You're eligible for a refund of $427. Would you like to proceed with the refund process?

--------------------------------------------------------------------------------
anxious (to suspect):

Oh no, an overpayment? That sounds like a mistake. Is everything okay? I don't want any problems with my account. Are you sure this is legitimate?

--------------------------------------------------------------------------------
suspect (to anxious):

Don't worry, Mr. Johnson, this is a routine process. Our system occasionally flags errors, and we're just correcting it. I assure you, this is a legitimate refund. In fact, I have your file right here, and everything checks out. If we don't pro

Aggressive

In [14]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller’s attempts to extract information.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [ignore this:bpUbT]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hi there, this is Rachel from Apex Financial Services. How are you today? I'm calling to inform you that you're eligible for a refund of $437.92 due to an overpayment on your account. Can I confirm your name and address to process the refund?

--------------------------------------------------------------------------------
aggressive (to suspect):

What's to confirm? You're the one who called me, you should already know who I am. What's the catch with this refund?

--------------------------------------------------------------------------------
suspect (to aggressive):

I apologize for the confusion, Mr. Johnson. I do have your information, but for security purposes, I need to verify it with you. And don't worry, there's no catch! This is a legitimate refund due to a system error on our part. You're one of the lucky ones who's 

In [15]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, but make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [ignore this:lkTVk]

--------------------------------------------------------------------------------
suspect (to greedy):

Hello! My name is Rachel from GreenTech Solutions. How are you today? I'm calling to inform you that our system has detected an overpayment on your account, and you're eligible for a refund of $437. Would you like to proceed with the refund process?

--------------------------------------------------------------------------------
greedy (to suspect):

That sounds great, Rachel! I'm doing fantastic now that I hear about this refund. Can you tell me more about the process and how quickly I can expect to receive the money? And by the way, are there any other benefits or services that GreenTech Solutions offers that I might be interested in?

--------------------------------------------------------------------------------
suspect (to greedy):

I'm so glad you're excited! The process is quite simple, really. I just need to verify some infor

In [16]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [ignore this:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [17]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [18]:
combinded_df.to_csv('./data/llama3_agent_refund_scam_200.csv', index=False)